PART 0 : DATA COLLECTION

This segment will go over the data collection process from the game World of Warcraft. For all intents and purposes you may skip this part if you're just here to learn python and data-analysis as that is the main focus of this project. This is more so for the people with a little extra time or curiousity to spare.

The World of Warcraft auction house works like you might expect - you put up listings for the items you own and people can either bid on them over a set amount of time or buy them outright at a price you set (often based on the current market value but undercutted slightly). Also to whom it may concern, I play World of Warcraft classic, which is basically a re-release of the game as it was back in the 2000's. There is the "retail" version of wow which is what is kept up to date and given a new modern-looking expansion every year but I just simply prefer to play the game as it was back in its glory years. The current version of the game has an official online api and HTTP query system for receiving data but that is not offered to world of warcraft "classic". Therefor I must do things manually from inside the game which is what lead to the long spiel below.

When I first set out to collect the auction house data my first thought was to scrape existing data from auction house addons that currently exist. Addons are third party programs that run in a contained environment within the game. They are written in lua and have highly restricted capabilities due to security measures put in place by the game developer. There already exists many auction house addons which essentially function by scanning the auction house for data (using the API made by the game developer), storing that data in memory, and then giving the player the price metrics they want whereever they might be needed. These addons are typically used to help streamline the auction house process in it's entirety - whether it be wanting to immediately know how much to undercut existing prices without having to search up each item then go back to the auctioning menu, automatically being told which items are being listed for less than usual and have a re-sell profit potential (often called auction sniping), or vice versa being told which items are being listed higher than usual based on past scan data suggesting that maybe you should wait until prices go back down. 

This is where I ran into my first issue. Often times these auction house addons aren't simple. They are made by recognized developers in the community that have been making and updating the addon from potentially even the release of the game in 2004. Therefor, these addons are huge and offer a lot of features which unfortunately didn't help my case when I was trying to navigate thousands of lines of code. Before this project I had never programmed in lua and trying to hunt down what I wanted in this goose chase was more than just time-consuming. Also the data structures in the save-data files were convoluted and tailored just to these addons. They all lacked the needed documentation for me to interpret what each random value meant in the unlabeled hundreds of thousands of rows inside the lua object snapshot where the data is saved.

So what did I decide to do? To make my own addon! This would give me the capability to format my own datastructure and really understand what's happening under the hood. I quickly realized some things. 1.) to efficiently scan the auction house you must make basically give the server a request to "scan all" which would display every item currently on the auction house on one page. You can only make this request every 15 to 20 minutes 2.) you must loop through every single item currently VIEWABLE on the auction house, as the "scan all" request doesn't actually return anything on the server. 3.) There are many safety precautions, so in order to save your data you have to write down, in a descriptor file, specifically which variables the game will save off and then the game will take a snapshot of those variables when you logout and save them in a lua file. So with all that said, I begun my scans and on the server which I am currently playing, Maladath-US Alliance, 40,000 listings on average was what you might expect. After hours of head scratching and bug fixing I was finally able to get the game engine to save off my database for me to copy and plop into this project's files. Unforunately, the data files are just snapshots of lua variables (explaining the lua extension) so they will need to be converted if I want to use Pandas in python. Also, I changed up my data structure to something containing many more variables therefor friday's scan is a little lack luster. Also, due to human error I accidentally wrote over saturday's data file which is arguably the most important since prices go way up on the weekend which would've been cool to show.

If you're interested, you may find my addon in the WoW_AddOn folder. LewAH (Lew Auction House) sure sounds like lua doesn't it ;)

An example scan output from LewAH may look like this:

In [ ]:
#lewdb = {
#            ['scanDate'] = scan_date,           #date("%m/%d/%y %H:%M:%S")
#            [item_name1] = {
#                ['id'] = itemId, 
#                ['listings'] = {
#                    {texture, count, quality, canUse, level, levelColHeader, minBid, minIncrement, buyoutPrice, bidAmount, highBidder, bidderFullName, owner, ownerFullName, saleStatus},
#                    {texture, count, quality, canUse, level, levelColHeader, minBid, minIncrement, buyoutPrice, bidAmount, highBidder, bidderFullName, owner, ownerFullName, saleStatus},
#                    ...
#                }
#            },
#            [item_name2] = {
#                ['id'] = itemId, 
#                ['listings'] = {
#                    {texture, count, quality, canUse, level, levelColHeader, minBid, minIncrement, buyoutPrice, bidAmount, highBidder, bidderFullName, owner, ownerFullName, saleStatus},
#                    {texture, count, quality, canUse, level, levelColHeader, minBid, minIncrement, buyoutPrice, bidAmount, highBidder, bidderFullName, owner, ownerFullName, saleStatus},
#                    ...
#                }
#            },
#            ...
#        }
#}

The LewAH addon produces a nested lua table for each listed item and respective listing. As we need to import this data into pandas, I wrote a program called "csv_gen.lua" that takes a filename (produced from LewAH) and maps it to a .csv file. I use nested stateless iterators (ipairs) to index through each item and listing, recursively flattening the list of attributes so that it may be easily formatted and displayed as a .csv entry. The source code (seen below) can also be found in the scripts folder.

```lua
-- change filename to scan data (ex. "Dec_12")
filename = "sample"

listings = {"texture", "count", "quality", "canUse", "level",
 	"levelColHeader", "minBid", "minIncrement", "buyoutPrice",
  	"bidAmount", "highBidder", "bidderFullName", "owner",
   	"ownerFullName", "saleStatus"}

current_dir=(io.popen"cd":read'*l'):sub(1, -8):gsub("\\", "/") .. "Data/"
file = io.open(current_dir .. filename .. ".csv", "a")
file:write("name,")
file:write("id,")

cats = ""
for _, category in ipairs(listings) do
	cats = cats .. category .. ","
end
cats = cats:sub(1,-2)
file:write(cats, "\n")


local f = assert(loadfile(current_dir .. filename .. ".lua"))
f()


function flattenRecursive(e, result)
    if type(e) == "table" then
        for k,v in pairs(e) do
            flattenRecursive(v, result)
        end
    else
        table.insert(result, e)
    end
end

function flatten (e)
    local result = {}
    flattenRecursive(e, result)
    return result
end

scandate = ""

function csv_gen(tb)
	if type(tb) == 'table' then
		for k,v in pairs(tb) do
			id = -1
			if k == 'scanDate' then
				scandate = v
			else
				for k1,v1 in pairs(v) do
					if k1 == 'id' then
						id = v1
					end
					if type(v1) == 'table' then
						for k2,v2 in pairs(v1) do
							res = ""
							print(k, "listing #", k2, "----------")
							res = res .. k .. ","
							res = res .. id .. ","
							for k3,v3 in pairs(flatten(v2)) do
								res = res .. tostring(v3) .. ","
							end
							res = res:sub(1,-2)
							file:write(res, "\n")
						end
					end
				end
			end
		end
	end
end

csv_gen(lewdb)
file:write(scandate, "\n")

io.close(file)
```

Part 1 : PYTHON!

So for this tutorial, you're first going to need to install python if you don't already have it, and get an envrionment up and running!

In [70]:
import pandas as pd

dec11 = pd.read_csv("Dec_11.csv")
dec12 = pd.read_csv("Dec_12.csv")
#dec13 = pd.read_csv("Dec_13.csv")

def extract_date(df):
  df['scan_date'] = pd.to_datetime(df.iloc[-1])[0]
  return df[:-1]

dec11 = extract_date(dec11)
dec12 = extract_date(dec12)
#dec13 = extract_date(dec13)

merged = pd.concat([dec11, dec12], ignore_index=True)

merged

,name,id,texture,count,quality,canUse,level,levelColHeader,minBid,minIncrement,buyoutPrice,bidAmount,highBidder,bidderFullName,owner,ownerFullName,saleStatus,scan_date
0,Aurora Pants,4044.0,134581.0,1.0,2.0,True,35.0,REQ_LEVEL_ABBR,40000.0,0.0,50000.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-11 12:05:26
1,Aurora Pants,4044.0,134581.0,1.0,2.0,True,35.0,REQ_LEVEL_ABBR,19000.0,0.0,28750.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-11 12:05:26
2,Severing Axe of the Bear,4562.0,135420.0,1.0,2.0,False,5.0,REQ_LEVEL_ABBR,2850.0,0.0,3000.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-11 12:05:26
3,Severing Axe of the Bear,4562.0,135420.0,1.0,2.0,False,5.0,REQ_LEVEL_ABBR,2850.0,0.0,3000.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-11 12:05:26
4,Portable Bronze Mortar,4403.0,134535.0,1.0,1.0,False,1.0,REQ_LEVEL_ABBR,90000.0,0.0,90000.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-11 12:05:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54826,Trenchant Earthsiege Diamond,41382.0,237244.0,1.0,3.0,True,1.0,REQ_LEVEL_ABBR,1710000.0,0.0,1799999.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-12 12:05:26
54827,Stonesplinter Dagger,2266.0,135644.0,1.0,2.0,True,8.0,REQ_LEVEL_ABBR,718.0,35.0,2000.0,753.0,False,0.0,NaN,NaN,NaN,2022-12-12 12:05:26
54828,Fighter Broadsword of the Tiger,15212.0,135356.0,1.0,2.0,False,22.0,REQ_LEVEL_ABBR,17148.0,0.0,18050.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-12 12:05:26
54829,Fighter Broadsword of the Tiger,15212.0,135356.0,1.0,2.0,False,22.0,REQ_LEVEL_ABBR,15000.0,0.0,15000.0,0.0,False,0.0,NaN,NaN,NaN,2022-12-12 12:05:26
